In this notebook we extract, process, and analyze TikTok video data from a JSON file, then predicts and ranks trending audios based on engagement metrics:
**1-Data Extraction:** Loads TikTok video data and extracts relevant fields (views, likes, shares, comments, hashtags, etc.).

**2-Sentiment Analysis:** Merges comment sentiment scores with video data.

**3-Trending Score Calculation:** Normalizes engagement metrics (likes, shares, play count, sentiment) and assigns weights to rank trending audios.

**4-Prediction Model:** Uses a Random Forest Regressor to forecast future trending audios.

**5-Audio Processing:** Downloads TikTok videos, extracts audio.

**6-Audio Generaion**:

transcript audio

generate text (using RAG)

convert generated text to speech

generate audio



In [1]:
import pandas as pd
import json

# Load the JSON file
file_path = "tiktok_data_trending_videos_with_comments.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract relevant fields with error handling
video_data = []
for video in data:
    # Initialize default values
    play_count = video.get("stats", {}).get("playCount", None)
    share_count = video.get("stats", {}).get("shareCount", None)
    comment_count = video.get("stats", {}).get("commentCount", None)
    digg_count = video.get("stats", {}).get("diggCount", None)
    collect_count = video.get("stats", {}).get("collectCount", None)
    follower_count = video.get("authorStats", {}).get("followerCount", None)
    heart_count = video.get("authorStats", {}).get("heartCount", None)
    video_count = video.get("authorStats", {}).get("videoCount", None)
    hashtags = [tag.get("hashtagName", "") for tag in video.get("textExtra", [])]


     # Extract music information
    music_info = video.get("music", {})
    music_title = music_info.get("title", None)
    music_author = music_info.get("authorName", None)
    music_id = music_info.get("id", None)
    music_play_url = music_info.get("playUrl", None)
    create_time = video.get("createTime", None)
    duration = music_info.get("duration", None)
    # Informations de l'auteur
    author_id= video.get("author", {}).get("uniqueId", "")
    username = video.get("author", {}).get("uniqueId", None)
    follower_count= video.get("authorStats", {}).get("followerCount", 0)
    heart_count=video.get("authorStats", {}).get("heartCount", 0)
    video_count= video.get("authorStats", {}).get("videoCount", 0)
    nickname =video.get("author", {}).get("nickname", "")
    video_count=video.get("authorStats", {}).get("videoCount", 0)
    friend_count=video.get("authorStats", {}).get("friendCount", 0)

    verified= video.get("author", {}).get("verified", False)
    # Extract video description
    description = video.get("desc", None)  # ✅ Added video description
     # Extract video URL from the nested video object
    video_url = None
    try:
        bitrate_info = video.get("video", {}).get("bitrateInfo", [])
        if bitrate_info:
            play_addr = bitrate_info[0].get("PlayAddr", {})
            url_list = play_addr.get("UrlList", [])
            if url_list:
                video_url = url_list[0]  # choose the first URL
    except Exception as e:
        video_url = None

    # Append to the list if 'play_count' is available
    if play_count is not None:
        video_info = {
            "id": video.get("id"),
            "play_count": play_count,
            "share_count": share_count,
            "comment_count": comment_count,
            "digg_count": digg_count,
            "collect_count": collect_count,
            "followers": follower_count,
            "likes": heart_count,
            "video_count": video_count,
            "hashtags": hashtags,
            "create_time": create_time,
            "duration": duration,
            "video_url": video_url,
            "music_title": music_title,
            "music_author": music_author,
            "music_id": music_id,
            "username": username,
            "music_play_url": music_play_url,
            "description": description,
            "author_id": author_id,
            "follower_count": follower_count,
            "heart_count": heart_count,
            "video_count": video_count,
            "nickname": nickname,
            "verified": verified,
            "video_count": video_count,
            "friend_count": friend_count

        }
        video_data.append(video_info)

# Convert to DataFrame
df = pd.DataFrame(video_data)
# Drop rows with missing values
df.dropna(inplace=True)

print(df.head())
print(df.columns)

                    id  play_count  share_count  comment_count  digg_count  \
0  7446111299458698498     4700000        54600           3247      885900   
1  7440845543267405111     2700000        30200           1508      348800   
2  7439015263565040904      799600        13100            468       49100   
3  7466606297304026373    46000000       983100          36900     7500000   
4  7467223106357169416     2800000         5091            482      138800   

   collect_count  followers     likes  video_count  \
0          88200     709600  30100000          945   
1          61300      15600    981900           16   
2           9300     147900   1800000          276   
3         366800     278100  22000000           55   
4          14000     781400  13700000          738   

                                            hashtags  ...  \
0                                                 []  ...   
1  [toji, tojifushiguro, jjk, jujutsukaisen, jjke...  ...   
2                      

In [2]:
import pandas as pd

data = pd.read_csv("comments_with_sentiments.csv")
print(data.columns)

video_sentiment_data = []

for index, row in data.iterrows():  # Use iterrows() to access rows as Series
    video_id = row['video_id']  # Access video_id by column name
    sentiment = row['sentiment']  # Access sentiment by column name
    video_sentiment_data.append([video_id, sentiment])

video_sentiment_df = pd.DataFrame(video_sentiment_data, columns=['video_id', 'sentiment'])
video_sentiment_df = video_sentiment_df.rename(columns={'video_id': 'id'})
print(video_sentiment_df.head())


Index(['video_id', 'comment', 'comment_count', 'comment_language',
       'translated_comment', 'cleaned_comment', 'sentiment', 'sentiment_b',
       'likes', 'replies', 'timestamp', 'date'],
      dtype='object')
                    id  sentiment
0  7446111299458698498     0.0000
1  7446111299458698498     0.2960
2  7446111299458698498     0.2732
3  7446111299458698498     0.0000
4  7446111299458698498     0.0000


In [3]:
# Convert the 'id' column in df to string
df['id'] = df['id'].astype(str)

# Convert the 'id' column in video_sentiment_df to string
video_sentiment_df['id'] = video_sentiment_df['id'].astype(str)
# Merge the first DataFrame with the video_sentiment_df on video_id
df = pd.merge(df, video_sentiment_df, on='id', how='left')

# Display the first few rows of the merged DataFrame
print(df.head())

                    id  play_count  share_count  comment_count  digg_count  \
0  7446111299458698498     4700000        54600           3247      885900   
1  7446111299458698498     4700000        54600           3247      885900   
2  7446111299458698498     4700000        54600           3247      885900   
3  7446111299458698498     4700000        54600           3247      885900   
4  7446111299458698498     4700000        54600           3247      885900   

   collect_count  followers     likes  video_count hashtags  ...    username  \
0          88200     709600  30100000          945       []  ...  notsaarita   
1          88200     709600  30100000          945       []  ...  notsaarita   
2          88200     709600  30100000          945       []  ...  notsaarita   
3          88200     709600  30100000          945       []  ...  notsaarita   
4          88200     709600  30100000          945       []  ...  notsaarita   

                                      music_play_u

In [4]:
!pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [5]:
df["tiktok_web_url"] = "https://www.tiktok.com/@" + df["username"] + "/video/" + df["id"]
#df["tiktok_web_url"][818]

In [6]:
!pip install pydub
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.5 MB/s eta 0:00:00


In [8]:
import os
import shutil

video_folder = "videos/"
audio_folder = "audios/"

# Clean the video folder
for filename in os.listdir(video_folder):
    file_path = os.path.join(video_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Clean the audio folder
for filename in os.listdir(audio_folder):
    file_path = os.path.join(audio_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [9]:
import yt_dlp
import os

failed_downloads = []  # Create a list to store failed URLs

def download_tiktok_video(video_url, output_path="videos/"):
    ydl_opts = {
        'outtmpl': f"{output_path}%(id)s.%(ext)s",
        'format': 'mp4'
    }
    video_id = None  # Initialize video_id to None
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(video_url, download=False)  # Get video info without downloading
            video_id = info_dict.get('id', None)  # Extract video ID
            ydl.download([video_url])  # Download the video
    except yt_dlp.utils.DownloadError as e:
        print(f"Error downloading {video_url}: {e}")
        failed_downloads.append(video_url)  # Add failed URL to the list

        # Delete the corresponding file if it exists
        if video_id:  # Now video_id is accessible here
            file_path = os.path.join(output_path, f"{video_id}.mp4")
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"Deleted failed download: {file_path}")

# Download videos
for url in df["tiktok_web_url"]:
    download_tiktok_video(url)


[TikTok] Extracting URL: https://www.tiktok.com/@notsaarita/video/7446111299458698498
[TikTok] 7446111299458698498: Downloading webpage
[TikTok] Extracting URL: https://www.tiktok.com/@notsaarita/video/7446111299458698498
[TikTok] 7446111299458698498: Downloading webpage
[info] 7446111299458698498: Downloading 1 format(s): bytevc1_1080p_1727287-1
[download] Destination: videos/7446111299458698498.mp4
[download] 100% of    3.09MiB in 00:00:00 at 3.23MiB/s   
[TikTok] Extracting URL: https://www.tiktok.com/@notsaarita/video/7446111299458698498
[TikTok] 7446111299458698498: Downloading webpage
[TikTok] Extracting URL: https://www.tiktok.com/@notsaarita/video/7446111299458698498
[TikTok] 7446111299458698498: Downloading webpage
[info] 7446111299458698498: Downloading 1 format(s): bytevc1_1080p_1727287-1
[download] videos/7446111299458698498.mp4 has already been downloaded
[download] 100% of    3.09MiB
[TikTok] Extracting URL: https://www.tiktok.com/@notsaarita/video/7446111299458698498
[Ti

ERROR: [TikTok] 7439015263565040904: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Error downloading https://www.tiktok.com/@baby111mama/video/7439015263565040904: ERROR: [TikTok] 7439015263565040904: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
[TikTok] Extracting URL: https://www.tiktok.com/@baby111mama/video/7439015263565040904
[TikTok] 7439015263565040904: Downloading webpage
[TikTok] Extracting URL: https://www.tiktok.com/@baby111mama/video/7439015263565040904
[TikTok] 7439015263565040904: Downloading webpage
[info] 7439015263565040904: Downloading 1 format(s): bytevc1_1080p_1216798-1
[download] videos/7439015263565040904.mp4 has already been downloaded
[download] 100% of    2.18MiB
[TikTok] Extracting URL: https://www.tiktok.com/@baby111mama/video/7439015263565040904
[TikTok] 7439015263565040904: Downloading webpage
[TikTok] Extracting URL: https://www.tiktok.com/@baby111mama/video/7439015263565040

ERROR: [TikTok] 7448500348471299349: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Error downloading https://www.tiktok.com/@khanh_ngoc236/video/7448500348471299349: ERROR: [TikTok] 7448500348471299349: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
[TikTok] Extracting URL: https://www.tiktok.com/@khanh_ngoc236/video/7448500348471299349
[TikTok] 7448500348471299349: Downloading webpage
[TikTok] Extracting URL: https://www.tiktok.com/@khanh_ngoc236/video/7448500348471299349
[TikTok] 7448500348471299349: Downloading webpage
[info] 7448500348471299349: Downloading 1 format(s): bytevc1_1080p_1183591-1
[download] videos/7448500348471299349.mp4 has already been downloaded
[download] 100% of    7.23MiB
[TikTok] Extracting URL: https://www.tiktok.com/@khanh_ngoc236/video/7448500348471299349
[TikTok] 7448500348471299349: Downloading webpage
[TikTok] Extracting URL: https://www.tiktok.com/@khanh_ngoc236/video/744850

...

Error downloading https://www.tiktok.com/@marie._2610/video/7464338768489614614: ERROR: [TikTok] 7464338768489614614: Video not available, status code 10204; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
[TikTok] Extracting URL: https://www.tiktok.com/@marie._2610/video/7464338768489614614
[TikTok] 7464338768489614614: Downloading webpage


ERROR: [TikTok] 7464338768489614614: Video not available, status code 10204; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Error downloading https://www.tiktok.com/@marie._2610/video/7464338768489614614: ERROR: [TikTok] 7464338768489614614: Video not available, status code 10204; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
[TikTok] Extracting URL: https://www.tiktok.com/@marie._2610/video/7464338768489614614
[TikTok] 7464338768489614614: Downloading webpage


ERROR: [TikTok] 7464338768489614614: Video not available, status code 10204; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Error downloading https://www.tiktok.com/@marie._2610/video/7464338768489614614: ERROR: [TikTok] 7464338768489614614: Video not available, status code 10204; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


In [ ]:
#remove the videos

In [10]:
# Filter the DataFrame
df_filtered = df[~df["tiktok_web_url"].isin(failed_downloads)]


In [11]:
from pydub import AudioSegment
import os

video_folder = "videos/"
audio_folder = "audios/"

# Ensure the audio folder exists
os.makedirs(audio_folder, exist_ok=True)

# Convert videos to audio
for file in os.listdir(video_folder):
    if file.endswith(".mp4"):  # Ensure it's a video file
        video_path = os.path.join(video_folder, file)
        audio_path = os.path.join(audio_folder, file.replace(".mp4", ".mp3"))

        # Extract audio
        audio = AudioSegment.from_file(video_path, format="mp4")
        audio.export(audio_path, format="mp3")
        print(f"Extracted: {audio_path}")


Extracted: audios/7464771808386993416.mp3
Extracted: audios/7461950035031346449.mp3
Extracted: audios/7460420296542014728.mp3
Extracted: audios/7465011342676626710.mp3
Extracted: audios/7467627972246244630.mp3
Extracted: audios/7457667992067804438.mp3
Extracted: audios/7466606297304026373.mp3
Extracted: audios/7467597244032814382.mp3
Extracted: audios/7445961651825773830.mp3
Extracted: audios/7465145660279459077.mp3
Extracted: audios/7465853964890803489.mp3
Extracted: audios/7459558497290554629.mp3
Extracted: audios/7463375325624388869.mp3
Extracted: audios/7467656692365380886.mp3
Extracted: audios/7438080729478614327.mp3
Extracted: audios/7452360885991001345.mp3
Extracted: audios/7456850959411498262.mp3
Extracted: audios/7467147165215755527.mp3
Extracted: audios/7463821836069981458.mp3
Extracted: audios/7468899308910153006.mp3
Extracted: audios/7467158390486142230.mp3
Extracted: audios/7464565111177383190.mp3
Extracted: audios/7468466741618724138.mp3
Extracted: audios/7459480867899985

In [ ]:
from pydub import AudioSegment
import os

audio_folder = "audios/"
audio_data = {}  # Dictionary to store audio data

for filename in os.listdir(audio_folder):
    if filename.endswith(".mp3"):  # Ensure it's an audio file
        audio_path = os.path.join(audio_folder, filename)
        try:
            audio = AudioSegment.from_file(audio_path, format="mp3")
            audio_data[filename] = audio  # Store audio with filename as key
        except Exception as e:
            print(f"Error loading audio from {audio_path}: {e}")

# Access audio data using filename
# Example: audio_data["my_audio.mp3"]

In [ ]:
df_filtered['audio_path'] = df_filtered['id'].apply(lambda x: f"audios/{x}.mp3" if f"{x}.mp3" in audio_data else None)

In [14]:
df_filtered.columns

Index(['id', 'play_count', 'share_count', 'comment_count', 'digg_count',
       'collect_count', 'followers', 'likes', 'video_count', 'hashtags',
       'create_time', 'duration', 'video_url', 'music_title', 'music_author',
       'music_id', 'username', 'music_play_url', 'description', 'author_id',
       'follower_count', 'heart_count', 'nickname', 'verified', 'friend_count',
       'sentiment', 'tiktok_web_url', 'audio_path'],
      dtype='object')

In [ ]:
df_filtered.ta

In [16]:
from google.colab import files
df_filtered.to_csv('my_dataframe.csv', index=False)  # Save to CSV
files.download('my_dataframe.csv')  # Download the file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# predict the top 10 trending audios based on post metrics (e.g., sentiment analysis score, likes, plays, shares, ...),

In [32]:
df=df_filtered

In [33]:
import pandas as pd

# Assuming your DataFrame is called `df`
# Normalize metrics (optional)
df['likes_norm'] = df['likes'] / df['likes'].max()
df['play_count_norm'] = df['play_count'] / df['play_count'].max()
df['share_count_norm'] = df['share_count'] / df['share_count'].max()
df['sentiment_norm'] = df['sentiment'] / df['sentiment'].max()

# Define weights for each metric
weights = {
    'likes': 0.4,
    'play_count': 0.3,
    'share_count': 0.2,
    'sentiment': 0.1
}

# Calculate trending score
df['trending_score'] = (
    weights['likes'] * df['likes_norm'] +
    weights['play_count'] * df['play_count_norm'] +
    weights['share_count'] * df['share_count_norm'] +
    weights['sentiment'] * df['sentiment_norm']
)

In [34]:
# Group by music_id or music_title and aggregate metrics
grouped_df = df.groupby('music_id').agg({
    'trending_score': 'sum',  # Sum of trending scores for each audio
    'likes': 'sum',           # Total likes for each audio
    'play_count': 'sum',       # Total plays for each audio
    'share_count': 'sum',      # Total shares for each audio
    'sentiment': 'mean'        # Average sentiment for each audio
}).reset_index()

In [35]:
# Sort by trending_score in descending order
grouped_df = grouped_df.sort_values(by='trending_score', ascending=False)

# Select the top 10 trending audios
top_10_trending_audios = grouped_df.head(10)

# Display the top 10 trending audios
print(top_10_trending_audios[['music_id', 'trending_score', 'likes', 'play_count', 'share_count', 'sentiment']])

                music_id  trending_score        likes  play_count  \
28   7374863020353833733       16.744027   7692000000  7704000000   
95   7457812956428897040       16.424351  70200000000   136500000   
94   7457236544798067489       12.947504  52000000000   468000000   
40   7438014659088337719       12.811911    432000000  2864000000   
93   7457235545281530646       11.717173   2028000000  6592000000   
130  7465780126882564910        9.593087  11436000000  3412000000   
57   7444352635198622465        9.490488   2035800000  4937400000   
144  7467951399825967889        8.581126  24244000000   952000000   
26   7366319142309333793        7.293668    496000000  4464000000   
103  7459480892824587013        7.289682   4256000000  2208000000   

     share_count  sentiment  
28      40000000   0.187170  
95         87009   0.155518  
94        337560   0.160858  
40     140000000   0.063687  
93       6252000   0.160270  
130     27192000   0.045802  
57      11165700   0.181823  


# Predict Future Trends

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Features and target
X = grouped_df[['likes', 'play_count', 'share_count', 'sentiment']]
y = grouped_df['trending_score']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict trending scores
grouped_df['predicted_trending_score'] = model.predict(X)

# Rank by predicted trending score
grouped_df = grouped_df.sort_values(by='predicted_trending_score', ascending=False)

# Display top 10 predicted trending audios
print(grouped_df.head(10))

                music_id  trending_score        likes  play_count  \
28   7374863020353833733       16.744027   7692000000  7704000000   
95   7457812956428897040       16.424351  70200000000   136500000   
94   7457236544798067489       12.947504  52000000000   468000000   
40   7438014659088337719       12.811911    432000000  2864000000   
93   7457235545281530646       11.717173   2028000000  6592000000   
57   7444352635198622465        9.490488   2035800000  4937400000   
103  7459480892824587013        7.289682   4256000000  2208000000   
130  7465780126882564910        9.593087  11436000000  3412000000   
26   7366319142309333793        7.293668    496000000  4464000000   
144  7467951399825967889        8.581126  24244000000   952000000   

     share_count  sentiment  predicted_trending_score  
28      40000000   0.187170                 14.222756  
95         87009   0.155518                 13.790472  
94        337560   0.160858                 11.836719  
40     140000000

In [48]:
#add audio_path
# 1. Create mapping
music_id_to_audio_path = df.set_index('music_id')['audio_path'].to_dict()

# 2. Add audio_path column
grouped_df['audio_path'] = grouped_df['music_id'].map(music_id_to_audio_path)

# Now group_df will have the 'audio_path' column
print(grouped_df.head())

               music_id  trending_score        likes  play_count  share_count  \
28  7374863020353833733       16.744027   7692000000  7704000000     40000000   
95  7457812956428897040       16.424351  70200000000   136500000        87009   
94  7457236544798067489       12.947504  52000000000   468000000       337560   
40  7438014659088337719       12.811911    432000000  2864000000    140000000   
93  7457235545281530646       11.717173   2028000000  6592000000      6252000   

    sentiment  predicted_trending_score                      audio_path  
28   0.187170                 14.222756  audios/7438631768900570401.mp3  
95   0.155518                 13.790472  audios/7457812915014356232.mp3  
94   0.160858                 11.836719  audios/7457236429052087584.mp3  
40   0.063687                 11.011050  audios/7467298521662655749.mp3  
93   0.160270                 10.340513  audios/7457235534713212182.mp3  


In [37]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2= r2_score(y_test, y_pred)
# Print the metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 0.3406086149130403
Mean Squared Error (MSE): 0.3828766684225118
Root Mean Squared Error (RMSE): 0.3828766684225118
R-squared (R²): 0.91964701767304


In [64]:

# Get top-predicted audios (based on predicted_trending_score)
top_predicted_audios = grouped_df.sort_values(by='predicted_trending_score', ascending=False).head(10)

print(top_predicted_audios)

                music_id  trending_score        likes  play_count  \
28   7374863020353833733       16.744027   7692000000  7704000000   
95   7457812956428897040       16.424351  70200000000   136500000   
94   7457236544798067489       12.947504  52000000000   468000000   
40   7438014659088337719       12.811911    432000000  2864000000   
93   7457235545281530646       11.717173   2028000000  6592000000   
57   7444352635198622465        9.490488   2035800000  4937400000   
103  7459480892824587013        7.289682   4256000000  2208000000   
130  7465780126882564910        9.593087  11436000000  3412000000   
26   7366319142309333793        7.293668    496000000  4464000000   
144  7467951399825967889        8.581126  24244000000   952000000   

     share_count  sentiment  predicted_trending_score  \
28      40000000   0.187170                 14.222756   
95         87009   0.155518                 13.790472   
94        337560   0.160858                 11.836719   
40     14000

# generate audios

In [45]:
!pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 8.2 MB/s eta 0:00:00


In [44]:
!pip install pydub

In [66]:
import os
from pydub import AudioSegment

# Create the "top10_audios" folder if it doesn't exist
top10_folder = "/content/top10_audios"
os.makedirs(top10_folder, exist_ok=True)

# Assuming 'grouped_df' is your DataFrame with 'audio_path' column

for index, row in top_predicted_audios.iterrows():
    audio_file = row['audio_path']

    # Load the audio file using pydub
    sound = AudioSegment.from_file(audio_file)

    # Extract the filename without extension
    filename_without_extension = os.path.basename(audio_file)[:-4]

    # Construct the new file path within "top10_audios" folder
    new_file_path = os.path.join(top10_folder, filename_without_extension + ".wav")

    # Export as WAV to the "top10_audios" folder
    sound.export(new_file_path, format="wav")
    print(f"Converted: {audio_file} to {new_file_path}")

Converted: audios/7438631768900570401.mp3 to /content/top10_audios/7438631768900570401.wav
Converted: audios/7457812915014356232.mp3 to /content/top10_audios/7457812915014356232.wav
Converted: audios/7457236429052087584.mp3 to /content/top10_audios/7457236429052087584.wav
Converted: audios/7467298521662655749.mp3 to /content/top10_audios/7467298521662655749.wav
Converted: audios/7457235534713212182.mp3 to /content/top10_audios/7457235534713212182.wav
Converted: audios/7444352536603053319.mp3 to /content/top10_audios/7444352536603053319.wav
Converted: audios/7459480867899985158.mp3 to /content/top10_audios/7459480867899985158.wav
Converted: audios/7465780031164402990.mp3 to /content/top10_audios/7465780031164402990.wav
Converted: audios/7455478531196243206.mp3 to /content/top10_audios/7455478531196243206.wav
Converted: audios/7467951360042913032.mp3 to /content/top10_audios/7467951360042913032.wav


In [57]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 824.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB

In [67]:
import whisper
import os
import pandas as pd

# Load the Whisper model
model = whisper.load_model("base")

# Directory containing audio files
audio_dir = "/content/top10_audios"  # Replace with your directory path

# Get list of audio files
audio_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith(('.mp3', '.wav', '.m4a'))]  # Include supported audio extensions

# Create empty lists to store data
file_paths = []
transcriptions = []

# Transcribe each audio file
for audio_path in audio_files:
    result = model.transcribe(audio_path)
    file_paths.append(audio_path)
    transcriptions.append(result['text'])

# Create DataFrame
df = pd.DataFrame({'file_path': file_paths, 'transcription': transcriptions})

# Save DataFrame to CSV (optional)
df.to_csv('transcriptions.csv', index=False)

# Display DataFrame
print(df)

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

                                       file_path  \
0  /content/top10_audios/7438631768900570401.wav   
1  /content/top10_audios/7457235534713212182.wav   
2  /content/top10_audios/7455478531196243206.wav   
3  /content/top10_audios/7459480867899985158.wav   
4  /content/top10_audios/7467951360042913032.wav   
5  /content/top10_audios/7465780031164402990.wav   
6  /content/top10_audios/7457236429052087584.wav   
7  /content/top10_audios/7444352536603053319.wav   
8  /content/top10_audios/7457812915014356232.wav   
9  /content/top10_audios/7467298521662655749.wav   

                                       transcription  
0   Tomo el ser a mi alivio, y hasta quién no se ...  
1   1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5...  
2  尿尿,尿尿 BOOO WRONG Möö Möö Möö Möö Mööö Möhohoho...  
3                                                     
4   Finally this shooting is getting better. Welc...  
5   Oh, shoot. Hey, man. Oh, hey, neighbor. Yeah,...  
6   дошло скинул и и Мод apocalypse С lâф 

In [68]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00


In [69]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.3 MB/s eta 0:00:00


In [70]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load the RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"  # RAG model fine-tuned on Natural Questions
tokenizer = RagTokenizer.from_pretrained(model_name)
retriever = RagRetriever.from_pretrained(model_name, index_name="exact", use_dummy_dataset=True)
model = RagSequenceForGeneration.from_pretrained(model_name, retriever=retriever)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

(…)ncoder_tokenizer%2Ftokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer%2Fvocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)oder_tokenizer%2Fspecial_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/10 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [79]:
# Process the first transcribed text

text = transcriptions[0]
# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

# Generate text using the RAG model
generated_ids = model.generate(inputs["input_ids"])
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(f"Input: {text}")
print(f"Generated Text: {generated_text}\\n")


Input:  Tomo el ser a mi alivio, y hasta quién no se asmía, no vivir en paz El conocí tu novio, prequenimo de imoso Y sé que no te quieres por su frío, va a hablar Además, tú no lo amas, porque no la la talla No seabe con placer de cómo lo haría yo Pero tenme paciencia, porque no es competencia Por eso me motivas, que hayo respetado Hoy se lo empezamos
Generated Text:  disavuna analogousitasjacudebrategest instant surpr aid resumedxsrov kineticformat tech dystop537\n


In [72]:
!pip install gtts

In [80]:
from gtts import gTTS
import os

# generated text to speech

# Create a gTTS object
tts = gTTS(text=generated_text)
# Create a unique filename for the audio file
audio_file = f"transcribed_audio_.mp3"

# Save the audio file
tts.save(audio_file)

print(f"Text-to-speech conversion saved to: {audio_file}")

#download the  audio to choose for the the new post
files.download(audio_file)

Text-to-speech conversion saved to: transcribed_audio_.mp3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>